In [1]:
CLUSTERS_YAML_PATH = "/home/deena_gergis/iti/iti_e2e_live/data/processed/features_skills_clusters_description.yaml"

TRACKING_URI = "file:///home/deena_gergis/iti/iti_e2e_live/notebooks/mlruns/"
# EXPERIMENT_NAME = "stackoverflow_single_model"

LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"

EXPERIMENT_ID = "1"
RUN_ID = "4c35a9c9d26a48e8bfccfae05a63d348"

In [2]:
import os 
import sklearn
import pickle
import yaml

import pandas as pd

import mlflow
from mlflow.tracking import MlflowClient

In [6]:
class JobPrediction:
    """Production Class for predicting the probability of a job from skills"""
    # Class attributes 
    path_clusters_config = None
    skills_clusters_df   = None
    
    tracking_uri  = None
    experiment_id = None
    run_id        = None
    
    model        = None
    all_features = None 
    all_jobs     = None 
    
    # Constructor
    def __init__(self, tracking_uri, experiment_id, run_id, clusters_yaml_path):
        # Store variables
        self.tracking_uri    = tracking_uri
        self.experiment_id = experiment_id
        self.run_id          = run_id
        
        # Retrieve model and features
        mlflow_objs = self.load_mlflow_objs(tracking_uri, 
                                            experiment_id, 
                                            run_id)
        self.model        = mlflow_objs[0]
        self.all_features = mlflow_objs[1]
        self.all_jobs     = mlflow_objs[2]
        
        # Load clusters config 
        self.path_clusters_config = clusters_yaml_path
        self.skills_clusters_df = self.load_clusters_config(clusters_yaml_path)
        
    # -------------------------------------------
    
    # Constructor helper functions 
    
    def load_mlflow_objs(self, tracking_uri, experiment_id, run_id):
        """Load objects from the MLflow run"""
        
        # Get artifact path
        artifact_path = os.path.join(tracking_uri.replace("file://", ""), 
                                     experiment_id, 
                                     run_id, 
                                     'artifacts')
        
        # Load data pkl
        data_path  = os.path.join(artifact_path, LOG_DATA_PKL)
        with open(data_path, 'rb') as handle:
            data_pkl = pickle.load(handle)
            
        # Load model pkl
        model_path = os.path.join(artifact_path, LOG_MODEL_PKL)
        with open(model_path, 'rb') as handle:
            model_pkl = pickle.load(handle)

        # Return model and data labels 
        return model_pkl["model_object"], data_pkl["features_names"], data_pkl["targets_names"]
    
    
    def load_clusters_config(self, path_clusters_config):
        """Load skills clusters developed in 03_feature_engineering.ipynb"""
        
        # Read YAML
        with open(path_clusters_config, "r") as stream:
            clusters_config = yaml.safe_load(stream)
            
        # Format into dataframe
        clusters_df = [(cluster_name, cluster_skill)
                       for cluster_name, cluster_skills in clusters_config.items()
                       for cluster_skill in cluster_skills]
        
        clusters_df = pd.DataFrame(clusters_df, 
                                   columns=["cluster_name", "skill"])
        return clusters_df

    
    # ========================================================
    # **************    Prediction Functions    **************  
    # ========================================================
    
    def create_features_array(self, available_skills):
        """Create the features array from a list of the available skills"""
        
        # Method's helper functions 
        def create_clusters_features(self, available_skills):
            sample_clusters = self.skills_clusters_df.copy()
            sample_clusters["available_skills"] = sample_clusters["skill"].isin(available_skills)
            cluster_features = sample_clusters.groupby("cluster_name")["available_skills"].sum()
            return cluster_features
            
        def create_skills_features(self, available_skills, exclude_features):
            all_features = pd.Series(self.all_features.copy())
            skills_names = all_features[~all_features.isin(exclude_features)]
            ohe_skills = pd.Series(skills_names.isin(available_skills).astype(int).tolist(), 
                                   index=skills_names)
            return ohe_skills
        
        # -------------------------
        
        # Method's main
        clusters_features = create_clusters_features(self, available_skills)
        skills_features   = create_skills_features(self, available_skills, 
                                                   exclude_features=clusters_features.index)
        # ... Combine features and sort 
        features = pd.concat([skills_features, clusters_features])
        features = features[self.all_features]
        return features.values 
    
    
    def predict_jobs_probabilities(self, available_skills):
        '''Returns probabilities of the different jobs according to the skills'''
        # Create features array 
        features_array = self.create_features_array(available_skills)
        
        # Predict and format
        predictions = self.model.predict_proba([features_array])
        predictions = [prob[0][1] for prob in predictions] # Keep positive probs 
        predictions = pd.Series(predictions, index=self.all_jobs)
        
        return predictions

    
    # ==============================================================
    

In [7]:
job_pred = JobPrediction(TRACKING_URI, EXPERIMENT_ID, RUN_ID, CLUSTERS_YAML_PATH)

In [8]:
job_pred.create_features_array(['Pandas', 'TensorFlow', 'Torch/PyTorch', 'Julia', 'Python'])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       3, 0, 0])

In [9]:
job_pred.predict_jobs_probabilities(['Pandas', 'TensorFlow', 'Torch/PyTorch', 'Julia', 'Python'])

Academic researcher                              0.543333
Data or business analyst                         0.020000
Data scientist or machine learning specialist    0.886667
Database administrator                           0.000000
DevOps specialist                                0.000000
Developer, QA or test                            0.020000
Developer, back-end                              0.020000
Developer, desktop or enterprise applications    0.010000
Developer, embedded applications or devices      0.010000
Developer, front-end                             0.010000
Developer, full-stack                            0.013333
Developer, game or graphics                      0.010000
Developer, mobile                                0.000000
Engineer, data                                   0.030000
Scientist                                        0.303333
System administrator                             0.010000
dtype: float64